# Check RMSE

In [30]:
# imports
from importlib import reload
import os 
import numpy as np
from scipy.sparse import csc_matrix

import h5py

from ulmo.mae import cutout_analysis
from ulmo.mae import mae_utils

import anly_rms

# Load up LLC

In [7]:
t,p = 10, 10
mae_valid_nonoise_tbl_file = 's3://llc/mae/Tables/MAE_LLC_valid_nonoise.parquet'

ogcm_path = os.getenv('OS_OGCM')
if ogcm_path is not None:
    enki_path = os.path.join(os.getenv('OS_OGCM'), 'LLC', 'Enki')
    local_mae_valid_nonoise_file = os.path.join(enki_path, 'PreProc', 'MAE_LLC_valid_nonoise_preproc.h5')


In [8]:
tbl_file = mae_valid_nonoise_tbl_file
recon_file = mae_utils.img_filename(t,p, local=True)
mask_file = mae_utils.mask_filename(t,p, local=True)
orig_file = local_mae_valid_nonoise_file

## Open up

In [11]:
f_orig = h5py.File(orig_file, 'r')
f_recon = h5py.File(recon_file, 'r')
f_mask = h5py.File(mask_file, 'r')

# Batch approach

In [80]:
reload(cutout_analysis)
rms = cutout_analysis.rms_images(f_orig, f_recon, f_mask, nimgs=100)

Loading images...
Masking edges
Calculate
Square
Mean
Root


In [71]:
rms

array([0.01208348, 0.0040086 , 0.01031118, 0.03580199, 0.02985872,
       0.01605023, 0.00310993, 0.0078866 , 0.00259255, 0.00899036,
       0.00623034, 0.00732086, 0.00188469, 0.00323403, 0.01437132,
       0.01202103, 0.0024626 , 0.00579177, 0.00349934, 0.00853155,
       0.00233403, 0.01781391, 0.00759246, 0.00557913, 0.00303644,
       0.03465322, 0.00301932, 0.03161074, 0.00723953, 0.00865494,
       0.00319845, 0.00634125, 0.00403289, 0.01085669, 0.00279277,
       0.00901101, 0.00743638, 0.00558569, 0.00852947, 0.01197581,
       0.00477717, 0.00277351, 0.04889103, 0.00301058, 0.03877452,
       0.00310707, 0.00209346, 0.00836256, 0.00267115, 0.00262845,
       0.00567396, 0.00305778, 0.00235853, 0.00337227, 0.00286731,
       0.00741211, 0.00607754, 0.01464252, 0.02242037, 0.0040077 ,
       0.0060764 , 0.00263646, 0.00685102, 0.00321847, 0.00505326,
       0.01964377, 0.02218987, 0.0179411 , 0.03840765, 0.02410337,
       0.02444774, 0.0058664 , 0.00241051, 0.00210187, 0.00409

# One by one approach

In [72]:
orig0 = f_orig['valid'][0,0,...]
recon0 = f_recon['valid'][0,0,...]
mask0 = f_mask['valid'][0,0,...]

In [73]:
reload(anly_rms)
rms0 = anly_rms.rms_single_img(orig0, recon0, mask0)
rms0

0.01208347991674546

In [79]:
one_rms = []
for idx in range(100):
    orig = f_orig['valid'][idx,0,...]
    recon = f_recon['valid'][idx,0,...]
    mask = f_mask['valid'][idx,0,...]    
    #
    irms = anly_rms.rms_single_img(orig, recon, mask)
    #print(f'{idx}: rms={rms}')
    one_rms.append(irms)
one_rms = np.array(one_rms)

In [82]:
np.all(np.isclose(rms, one_rms))

True

----

# Debuggin

## X

In [68]:
patch_sz = 4
nimgs = 2

# Grab em
orig_imgs = f_orig['valid'][:nimgs,0,...]
recon_imgs = f_recon['valid'][:nimgs,0,...]
mask_imgs = f_mask['valid'][:nimgs,0,...]

# Mask out edges
print("Masking edges")
mask_imgs[:, 0:patch_sz, :] = 0
mask_imgs[:, -patch_sz:, :] = 0
mask_imgs[:, :, 0:patch_sz] = 0
mask_imgs[:, :, -patch_sz:] = 0

# Analyze
print("Calculate")
calc = (orig_imgs - recon_imgs)*mask_imgs

# Square
print("Square")
calc = calc**2

# Mean
print("Mean")
nmask = np.sum(mask_imgs, axis=(1,2))
calc = np.sum(calc, axis=(1,2)) / nmask

# RMS
print("Root")
rmss = np.sqrt(calc)

Masking edges
Calculate
Square
Mean
Root


In [69]:
rmss

array([0.01208348, 0.0040086 ], dtype=float32)

In [28]:
nmask

array([128.,  64.], dtype=float32)

## Single

In [45]:
mask_imgs.shape

(2, 64, 64)

In [31]:
mask_sparse = csc_matrix(mask_imgs[0,...])
mask_i,mask_j = mask_sparse.nonzero()

In [32]:
len(mask_i)

128

In [33]:
orig0.shape

(64, 64)

In [64]:
orig_img  = orig0[4:-4, 4:-4]
recon_img = recon0[4:-4, 4:-4]
mask_img  = mask0[4:-4, 4:-4]

# Find i,j positions from mask
mask_sparse = csc_matrix(mask_img)
mask_i,mask_j = mask_sparse.nonzero()

# Find differences
diff = np.zeros(len(mask_i))
for idx, (i, j) in enumerate(zip(mask_i, mask_j)):
    diff[idx] = orig_img[i,j] - recon_img[i,j]

In [67]:
tmp = mask_imgs[0,4:-4,4:-4]
np.all(tmp == mask0)

/tmp/ipykernel_151583/3657733041.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.all(tmp == mask0)


False

In [48]:
mask_i

array([ 8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9, 10,
       10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12,
       12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14,
       14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21,
       21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24,
       25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 27,
       27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30,
       30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33,
       33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35,
       35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 38, 38, 38, 38, 39,
       39, 39, 39, 40, 40

In [49]:
diff.size

288

In [52]:
np.sum(mask0), 0.1 * 64**2

(416.0, 409.6)

In [53]:
np.sum(mask_imgs[0,...])

128.0

In [54]:
mask_imgs[0,...]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)

In [58]:
np.all(mask0 == mask_imgs[0,...])

False

In [57]:
64**2

4096

In [60]:
mask0 = f_mask['valid'][0,0,...]
mask0.shape

(64, 64)

In [61]:
mask_imgs = f_mask['valid'][:nimgs,0,...]
mask_imgs.shape

(2, 64, 64)

In [62]:
np.all(mask0 == mask_imgs[0,...])

True